In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('IN_emissions_data.csv')

In [3]:
cols = list(data.columns)

In [4]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
scaled_df = pd.DataFrame(data_scaled, columns=cols)

In [7]:
scaled_df['cover_crop_dummy'] = scaled_df['IN Corn Cover Crop (Thousands of acres)'] != 0

In [8]:
y = scaled_df.pop('cover_crop_dummy')
X = scaled_df

In [9]:
len(X)

28

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=19)

In [11]:
hold_out = X_test.drop('efficiency', axis=1)

In [12]:
from imblearn.over_sampling import SMOTENC

Bootstrap

In [13]:
len(X_train)

22

In [14]:
bootstrap_data = X_train.join(pd.DataFrame(y_train))

In [15]:
size = int(len(X_train))
r_state = 19
while size < 100:
    random_sample = bootstrap_data.sample(n=7, replace=True, random_state=r_state)
    bootstrap_data = pd.concat([bootstrap_data, random_sample])
    size = len(bootstrap_data)
    r_state = r_state + 1

In [16]:
r_state

31

In [17]:
print(bootstrap_data.info())
print(r_state)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 3 to 9
Data columns (total 14 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Year                                     106 non-null    float64
 1   Per Acre Emissions                       106 non-null    float64
 2   N                                        106 non-null    float64
 3   P                                        106 non-null    float64
 4   Potash                                   106 non-null    float64
 5   IN Corn Yield per Acre                   106 non-null    float64
 6   Precipitation                            106 non-null    float64
 7   Average Temperature                      106 non-null    float64
 8   IN No Till Corn (Thousands of acres)     106 non-null    float64
 9   IN Con Till Corn (Thousands of acres)    106 non-null    float64
 10  IN Corn Cover Crop (Thousands of acres)  106 non-nul

SMOTE

In [18]:
bootstrap_y = bootstrap_data.pop('cover_crop_dummy')
bootstrap_X = bootstrap_data

In [19]:
bootstrap_X[(bootstrap_X['Per Acre Emissions'] >= 2.87) & (bootstrap_X['Per Acre Emissions'] <= 2.88)]

,Year,Per Acre Emissions,N,P,Potash,IN Corn Yield per Acre,Precipitation,Average Temperature,IN No Till Corn (Thousands of acres),IN Con Till Corn (Thousands of acres),IN Corn Cover Crop (Thousands of acres),Conventional Till,efficiency


In [20]:
sm = SMOTENC(categorical_features=[1], random_state=19)

In [21]:
X_train_smote, y_train_smote = sm.fit_resample(bootstrap_X, bootstrap_y) 

ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

In [ ]:
len(X_train_smote[(X_train_smote['Per Acre Emissions'] >= 2.87) & (X_train_smote['Per Acre Emissions'] <= 2.88)])

In [ ]:
sns.scatterplot(X_train_smote['Per Acre Emissions'], X_train_smote['IN Corn Yield per Acre'])

In [ ]:
sns.scatterplot(data['Per Acre Emissions'], data['IN Corn Yield per Acre'])

In [ ]:
smote_data = X_train_smote.drop('efficiency', axis=1)

In [ ]:
smote_data.to_csv('/Users/coleromanyk/Documents/GitHub/Capstone 2/SMOTE_data.csv', index=False)
hold_out.to_csv('/Users/coleromanyk/Documents/GitHub/Capstone 2/hold_out_data.csv', index=False)